In [19]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, register, execute, get_backend
from qiskit.tools.visualization import circuit_drawer, plot_histogram, plot_state
from qiskit.tools.qi.qi import state_fidelity
import random, pickle, time, datetime, xlsxwriter
import numpy as np
import scipy.io as spio
import importlib
import circuit_builder
importlib.reload(circuit_builder)
import encoding_schemes
importlib.reload(encoding_schemes)

#setup
import sys, getpass
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
    print()
except:
    APItoken = getpass.getpass('Please input your token and hit enter: ')
    qx_config = {
        "APItoken": APItoken,
        "url":"https://quantumexperience.ng.bluemix.net/api"}
    print('Qconfig.py not found in qiskit-tutorial directory; Qconfig loaded using user input.')
    
#matlab stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import matrix
from math import pi

#set 
QX_URL = "https://quantumexperience.ng.bluemix.net/api"
QX_TOKEN = "06704f99fc6dd68ab5ae2663f0861723406de8b4553cacdabdf500292c937717442daa90c2ffa19109cfb9114adb7c87711a08860d34d1da9332ac664e4e24a7"

#register token and url
register(QX_TOKEN, QX_URL)

# query for available backends
print("Available backends: ", available_backends())

backend = "local_qasm_simulator"

# This function deals with the case when circuits contains a length one circuit, which is parsed
# as a string rather than an array. We convert all strings into arrays so getShortest works properly.
def clean(circuits):
    strings = []
    newArray = []
    for element in circuits:
        if isinstance(element,np.ndarray):
            newArray.append(element)
            continue
        else:
            strings.append(element)
    strings = np.asanyarray(strings)
    newArray = np.append(newArray, strings)
    for string in strings:
        string = np.asanyarray([string])
        print(string)
        newArray = np.append(newArray,string)
    print()
    print(newArray)
    print()
    return newArray
        

# This function is not robust. Ties between circuits of equal length are broken arbitrarily.
# Sometimes, it may matter, because two implementations that are the same length may involve
# different numbers of gates and may have different performances. However, from current data,
# it seems that the shortest circuit is usually unique, so this is not currently a concern.
def getShortest(circuits):
    clean(circuits)
    circuit = min(circuits, key = len)
    return [circuit]

# code for extracting gate constructions
extract = spio.loadmat('shortnormgates.mat', squeeze_me = True)
P = extract['CNOT21norm']


# IMPORTANT SETTINGS
#
# Update the scheme to reflect your encoding scheme
scheme = encoding_schemes.Uncoded()

# Update desired outputs to reflect your circuit
desiredOutputs = [["0000","1111"], ["0101","1010"],  ["0110","1001"], ["1100","0011"]]
#desiredOutputs = [["00"], ["10"], ["01"], ["11"]]

# Setup variables
initialStates = [2,3]
reps = 9

# Specify circuits
circuits = CNOT
circuits = getShortest(circuits)
print(circuits)
# Try this one (replace CZ as appropriate) if you only want to find the desiredOutputs for a given circuit
# circuits = [CZ[0]]
#


# These update automatically with scheme
n = scheme.n
validOutputs = scheme.validOutputs

startTime = time.time()

# Main benchmarking procedure
for circuit in circuits:
    
    for compiler in initialStates:
        print("Circuit is: "+str(circuit))
        print("Compiler is: "+str(compiler))
            
        avgSuccess = 0

        for trial in range(reps):
            
            # Standard setup
            qr = QuantumRegister(n)
            cr = ClassicalRegister(n)
            qc = QuantumCircuit(qr,cr)
            
            scheme.prepareZeros(qc)
            
            # Maps the input info into actual physical qubits
            perm = scheme.perm
            scheme.setPerm()
            
            # Sets our initial state
            scheme.setInput(qc, compiler)
            
            
            # Build the circuit
            registerPerm = circuit_builder.buildCirc(qc, circuit, perm)
            
            for i in range(n):
                qc.measure(qr[registerPerm[i]],cr[i])
            
            # Standard execution code
            job = execute(qc, backend)
            #Print Job Status 
            lapse = 0
            interval = 120
            while not job.done:
                print('Status @ {} seconds'.format(interval * lapse))
                print(job.status)
                time.sleep(interval)
                lapse += 1
            print(job.status)
        
            #get Results
            result = job.result()
            data = result.get_counts(qc)
            
            
            
            
            # Prints raw data
            print(data)
            
            # Clean and process data
            successes = 0
            totalValid = 0
            for key, value in data.items():
                if key in desiredOutputs[compiler]:
                    successes += value
                if key in validOutputs:
                    totalValid += value
            
            # If we hit this, something went very wrong
            if totalValid == 0:
                print("ERROR WITH VALID INPUTS")
                
            else:
                # Comment out if running more trials
                # print("Success rate is: "+ str(successes/totalValid))
                avgSuccess += successes/totalValid
        
        # Calculate and display information
        avgSuccess /= reps
        print("Average of success rates is: "+ str(avgSuccess)+ "\n")

endTime = time.time()
print("Time running: "+str(endTime-startTime))

Qconfig loaded from C:\Users\ddrsq\OneDrive\IBM Q\qecc_benchmark\Qconfig.py.



Got a 502 code response to https://quantumexperience.ng.bluemix.net/api/Backends?access_token=PqNBb3ngCD1AvVa9NqClzldLtKKr0P2alZHxYbSCdTlOzjuKSjgDRgK23mcW3a1o: 502 Bad Gateway: Registered endpoint failed to handle the request.



{'job_id': '5b51fa4811322d003bf0fd09', 'status': <JobStatus.DONE: 'job has successfully run'>, 'status_msg': 'job has successfully run'}
{'00': 137, '01': 856, '10': 14, '11': 17}
Status @ 0 seconds
{'job_id': None, 'status': <JobStatus.INITIALIZING: 'job is being initialized'>, 'status_msg': 'Job is initializing. Please, wait a moment.'}
Status @ 120 seconds
{'job_id': '5b51fc2e1deae90038496c7a', 'status': <JobStatus.QUEUED: 'job is queued'>, 'status_msg': 'job is queued', 'queue_position': 3}
{'job_id': '5b51fc2e1deae90038496c7a', 'status': <JobStatus.DONE: 'job has successfully run'>, 'status_msg': 'job has successfully run'}
{'00': 156, '01': 847, '10': 9, '11': 12}
Status @ 0 seconds
{'job_id': None, 'status': <JobStatus.INITIALIZING: 'job is being initialized'>, 'status_msg': 'Job is initializing. Please, wait a moment.'}
Status @ 120 seconds
{'job_id': '5b51fd20a90c3f003f7e5199', 'status': <JobStatus.QUEUED: 'job is queued'>, 'status_msg': 'job is queued', 'queue_position': 3}
S